In [1]:
import os
os.chdir("../")

In [2]:
pwd

'/Users/sujith/Desktop/Files/projects/QuickGist-A-Text-Summarizer-tool'

In [34]:
from dataclasses import dataclass #used for creating entities
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    s3_bucket: str
    s3_dataset: str

In [35]:
from src.code.constants import *
from src.code.utils.common import read_yaml, create_directories


In [36]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        keys_filepath = KEYS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.keys = read_yaml(keys_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.dataset

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir ,
            s3_bucket=config.s3_bucket,
            s3_dataset=config.s3_dataset
        )
        return data_ingestion_config
    
    def get_keys(self):
        keys=self.keys
        return keys

In [37]:
import os
import urllib.request as request
import zipfile
from src.code.logging import LogTool
from src.code.utils.common import get_size
import boto3
from botocore.exceptions import NoCredentialsError

In [38]:
class DataIngestion:
    def __init__(self, config, keys):
        self.config = config
        self.keys = keys

    def download_file_from_s3(self):
        if not os.path.exists(self.config.local_data_file):
            self._download_file_from_s3()
        else:
            LogTool.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def _download_file_from_s3(self):
        try:
            s3 = boto3.client(
                's3',
                aws_access_key_id=self.keys.aws_access_key_id,
                aws_secret_access_key=self.keys.aws_secret_access_key
            )

            s3.download_file(self.config.s3_bucket, self.config.s3_dataset, self.config.local_data_file)

            LogTool.info(f"{self.config.local_data_file} downloaded from S3!")

        except NoCredentialsError:
            LogTool.error("Credentials not available. Unable to download file from S3.")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [40]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    aws_keys=config.get_keys()
    data_ingestion = DataIngestion(config=data_ingestion_config,keys=aws_keys)
    data_ingestion.download_file_from_s3()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-10 18:19:13,361: INFO: common] yaml file: configs/config.yaml loaded successfully
[2024-01-10 18:19:13,363: INFO: common] yaml file: param.yaml loaded successfully
[2024-01-10 18:19:13,364: INFO: common] yaml file: configs/aws_keys.yaml loaded successfully
[2024-01-10 18:19:13,364: INFO: common] created directory at: artifacts
[2024-01-10 18:19:13,365: INFO: common] created directory at: artifacts/dataset
[2024-01-10 18:20:18,705: INFO: 2198715394] artifacts/dataset/dataset.zip downloaded from S3!
